# Desmascarando Robôs
### CRISP-DM Cycle 1
---

Imagine um mercado online, um palco digital onde diversos leilões se desenrolam a cada segundo. Neste ambiente, participantes do mundo inteiram lançam seus lances em busca de objetos desejados, desde joias até equipamentos tecnológicos. No entanto, nem todos os jogadores neste campo são humanos; alguns são robôs programados para manipular os resultados dos leilões.

Seu desafio é se aprofundar nesses dados, explorar as camadas de atividade nos leilões e conseguir construir um modelo que saiba muito bem diferenciar humanos de robôs.

> Disclaimer: This is a fictional bussiness cas

## 0. PREPARATION

### 0.1 Settings

In [1]:
# Settings imports
import os
import sys
import pandas as pd
from dotenv import load_dotenv

# Load .env file
env_path = "../.env"
load_dotenv(dotenv_path=env_path)

# Seed
seed = int(os.getenv("SEED"))

# Add path
path = os.getenv("HOMEPATH")

# Add path to sys.path
sys.path.append(path)

In [2]:
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

### 0.2 Data

**Train e Test**

- **id_participante**: Identificador único do participante
- **conta_pagamento**: Conta de pagamento associada ao participante (com o valor ocultado) # Não será utilizada
- **endereco**: Endereço postal do participante # Não será utilizada
- **resultado**: A variável alvo que identifica se o participante é um robô ou um humano. (Robô = 1 e Humano = 0). (*target*)

- **Robôs Confirmados**: Participantes com provas claras de atividades fraudulentas, resultando em banimento da plataforma. São rotulados como robôs no conjunto de dados (resultado = 1).

- **Robôs Suspeitos**: Participantes com atividades atípicas ou estatísticas que superam a média, mas sem provas definitivas de fraude. A classificação deles como robôs é incerta.

**Lances**

- **id_lance**: Identificador único do lance
- **id_participante**: Identificador único do participante
- **leilao**: Identificador único do leilão 
- **mercadoria**: A categoria da mercadoria leiloada
- **dispositivo**: O dispositivo utilizado pelo visitante
- **tempo**: O tempo que o lance foi feito
- **pais**: O país que o IP pertence
- **ip**: O IP do participante
- **url**: A URL de onde o participante foi referido

## 1. Modelling

### 1.1 Loading Data

In [3]:
X_train = pd.read_feather(path + "/data/processed/X_train.feather")
X_test = pd.read_feather(path + "/data/processed/X_test.feather")
X_val = pd.read_feather(path + "/data/processed/X_val.feather")


y_train = pd.read_pickle(path + "/data/processed/y_train.pkl")
y_test = pd.read_pickle(path + "/data/processed/y_test.pkl")
y_val = pd.read_pickle(path + "/data/processed/y_val.pkl")

print(len(X_train.index))

253209


In [ ]:
from sklearn.pipeline import Pipeline
from helper.classes.Pipeline import MLPipeline


feature_transformations = {
    "log": [
        "contagem_participante",
        "contagem_leilao",
        "contagem_conta_pagamento",
        "frequencia_dispositivo",
    ],
    "one_hot": [
        "dispositivo",
        "leilao",
        "periodo_dia",
        "mercadoria",
        "conta_pagamento",
    ],
    "ordinal": ["ip_classe"],
    "hashing": ["pais", "url", "endereco"],
    "min_max_scaler": [
        "hora_sin",
        "hora_cos",
        "minuto_sin",
        "minuto_cos",
        "segundo_sin",
        "segundo_cos",
    ],
    "robust_scaler": ["hora", "minuto", "segundo"],
}

In [ ]:
proportion = float(len(y_train[y_train == 0])) / len(y_train[y_train == 1])
class_weights = {0: 0.87, 1: 0.13}

models = [
    # KNeighborsClassifier(n_neighbors=5),
    # LogisticRegression(class_weight=class_weights, random_state=seed),
    # DecisionTreeClassifier(class_weight=class_weights, random_state=seed),
    RandomForestClassifier(class_weight=class_weights, random_state=seed),
    # AdaBoostClassifier(random_state=seed),
    # GradientBoostingClassifier(random_state=seed),
    # ExtraTreesClassifier(class_weight=class_weights, random_state=seed),
    # XGBClassifier(
    #    scale_pos_weight=proportion,
    #    objective="binary:logistic",
    #    eval_metric="logloss",
    #    random_state=seed,
    # ),
    # LGBMClassifier(
    #    is_unbalance=True,
    #    objective="binary",
    #    metric="binary_logloss",
    #    random_state=seed,
    # ),
    # CatBoostClassifier(scale_pos_weight=proportion, random_state=seed),
]

In [ ]:
print(len(X_train.index))
print(len(y_train.index))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, FunctionTransformer
)
from category_encoders import OrdinalEncoder, HashingEncoder
from sklearn.compose import ColumnTransformer
import warnings

warnings.filterwarnings('ignore')

def test_transformations(X, feature_transformations):
    transformers = {
        'log': FunctionTransformer(np.log1p, validate=True),
        'one_hot': OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
        'hashing': HashingEncoder(),
        'ordinal': OrdinalEncoder(),
        'standard_scaler': StandardScaler(),
        'min_max_scaler': MinMaxScaler(),
        'robust_scaler': RobustScaler()
    }
    
    for transformer_name, features in feature_transformations.items():
        print(f"Testing {transformer_name} transformation on features: {features}")
        transformer = transformers[transformer_name]
        try:
            transformed = transformer.fit_transform(X[features])
            print(f"Transformation {transformer_name} succeeded. Output shape: {transformed.shape}")
        except Exception as e:
            print(f"Transformation {transformer_name} failed with error: {e}")
            
test_transformations(X_train, feature_transformations)

In [ ]:
pipeline = MLPipeline(feature_transformations, models)
resultados = pipeline.run(X_train, y_train, X_val, y_val)
resultados

In [ ]:
# pipeline = MLPipeline(X_train, y_train, models)
# pipeline.build_pipeline(
#    log_list=log_list,
#    ohe_list=ohe_list,
#    ordinal_list=ordinal_list,
#    robust_scaler_list=robust_scaler_list,
#    min_max_scaler_list=min_max_scaler_list,
#    standard_scaler_list=standard_scaler_list,
# )

# Treinar e avaliar modelos
# evaluation_results = pipeline.train_and_evaluate_models(X_val, y_val)
# evaluation_results

In [ ]:
# Para Cross-Validation
# pipeline_cv = MLPipelineCV(X_train, y_train, models)
# pipeline_cv.build_pipeline(log_list, ohe_list, catboost_list, robust_scaler_list, min_max_scaler_list, standard_scaler_list)

# Treinar e avaliar modelos com Cross-Validation
# cv_results = pipeline_cv.train_and_evaluate_cv()
# cv_results